In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[2] += v[2]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b and x[2]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b and x[2]<b:
            ch = 1.0
            stop = True
        elif x[2]>= b and x[0]<b and x[1]<b:
            ch = 2.0
            stop = True
        elif (x[0]>= b and x[1]>=b) or (x[1]>= b and x[2]>=b) or (x[0]>= b and x[2]>=b):
            rt = 0
            x = np.array([0.0, 0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Kr2011_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [4]:
tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(tr_df[['rt', 'choice_0', 'choice_1', 'choice_2', 'sbj']])
X_train = np.asarray(tr_df[['item_value_0', 'item_value_1', 'item_value_2', 
                            'gaze_0', 'gaze_1', 'gaze_2']])

obs_test = np.asarray(ts_df[['rt', 'choice_0', 'choice_1', 'choice_2', 'sbj']])
X_test = np.asarray(ts_df[['item_value_0', 'item_value_1', 'item_value_2', 
                           'gaze_0', 'gaze_1', 'gaze_2']])

print(X_train.shape, X_test.shape)

(2922, 6) (2922, 6)


In [5]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)


def G2(data, pred):
    
    sbj_indices = tf.cast(data[:, 4], tf.int32)
    
    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))
    
    ll1 = (obs_train[:, 1]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 1])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]))
    ll2 = (obs_train[:, 2]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 0])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]))
    ll3 = (obs_train[:, 3]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 0])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 1]))
    
    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2 + ll3, 1e-14)), axis=-1)

In [6]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=3, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [7]:
layers_neuron = [10, 10, 10, 10]
input_shape = 6
output_shape = 3
activation = 'sigmoid'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                70        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                                 
 out_layer (Out_layer)       (None, 3)                 123       
                                                                 
Total params: 523 (2.04 KB)
Trainable params: 523 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
sbj_indices = tf.cast(obs_train[:, 4], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [9]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 835ms/step - loss: 28592.6934
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 28389.4219
Epoch 3/5000
1/1 [==============================] - 0s 2ms/step - loss: 28245.6035
Epoch 4/5000
1/1 [==============================] - 0s 2ms/step - loss: 28127.1758
Epoch 5/5000
1/1 [==============================] - 0s 9ms/step - loss: 28023.4277
Epoch 6/5000
1/1 [==============================] - 0s 4ms/step - loss: 27929.3906
Epoch 7/5000
1/1 [==============================] - 0s 4ms/step - loss: 27842.3086
Epoch 8/5000
1/1 [==============================] - 0s 3ms/step - loss: 27760.4590
Epoch 9/5000
1/1 [==============================] - 0s 3ms/step - loss: 27682.6836
Epoch 10/5000
1/1 [==============================] - 0s 3ms/step - loss: 27608.1719
Epoch 11/5000
1/1 [==============================] - 0s 4ms/step - loss: 27536.3125
Epoch 12/5000
1/1 [==============================] - 0s 3ms/step - loss: 27466.6445

1/1 [==============================] - 0s 3ms/step - loss: 23288.8047
Epoch 99/5000
1/1 [==============================] - 0s 3ms/step - loss: 23251.6367
Epoch 100/5000
1/1 [==============================] - 0s 3ms/step - loss: 23214.7031
Epoch 101/5000
1/1 [==============================] - 0s 4ms/step - loss: 23178.0039
Epoch 102/5000
1/1 [==============================] - 0s 3ms/step - loss: 23141.5352
Epoch 103/5000
1/1 [==============================] - 0s 3ms/step - loss: 23105.3008
Epoch 104/5000
1/1 [==============================] - 0s 3ms/step - loss: 23069.2949
Epoch 105/5000
1/1 [==============================] - 0s 3ms/step - loss: 23033.5215
Epoch 106/5000
1/1 [==============================] - 0s 3ms/step - loss: 22997.9785
Epoch 107/5000
1/1 [==============================] - 0s 3ms/step - loss: 22962.6680
Epoch 108/5000
1/1 [==============================] - 0s 3ms/step - loss: 22927.5820
Epoch 109/5000
1/1 [==============================] - 0s 3ms/step - loss: 22892.7

1/1 [==============================] - 0s 3ms/step - loss: 20639.5195
Epoch 195/5000
1/1 [==============================] - 0s 3ms/step - loss: 20619.7773
Epoch 196/5000
1/1 [==============================] - 0s 2ms/step - loss: 20600.1562
Epoch 197/5000
1/1 [==============================] - 0s 4ms/step - loss: 20580.6582
Epoch 198/5000
1/1 [==============================] - 0s 3ms/step - loss: 20561.2773
Epoch 199/5000
1/1 [==============================] - 0s 3ms/step - loss: 20542.0156
Epoch 200/5000
1/1 [==============================] - 0s 3ms/step - loss: 20522.8672
Epoch 201/5000
1/1 [==============================] - 0s 3ms/step - loss: 20503.8301
Epoch 202/5000
1/1 [==============================] - 0s 3ms/step - loss: 20484.9141
Epoch 203/5000
1/1 [==============================] - 0s 3ms/step - loss: 20466.1113
Epoch 204/5000
1/1 [==============================] - 0s 3ms/step - loss: 20447.4180
Epoch 205/5000
1/1 [==============================] - 0s 3ms/step - loss: 20428.

1/1 [==============================] - 0s 3ms/step - loss: 19156.2852
Epoch 291/5000
1/1 [==============================] - 0s 4ms/step - loss: 19143.9805
Epoch 292/5000
1/1 [==============================] - 0s 3ms/step - loss: 19131.7246
Epoch 293/5000
1/1 [==============================] - 0s 3ms/step - loss: 19119.5098
Epoch 294/5000
1/1 [==============================] - 0s 3ms/step - loss: 19107.3379
Epoch 295/5000
1/1 [==============================] - 0s 4ms/step - loss: 19095.2129
Epoch 296/5000
1/1 [==============================] - 0s 3ms/step - loss: 19083.1270
Epoch 297/5000
1/1 [==============================] - 0s 3ms/step - loss: 19071.0859
Epoch 298/5000
1/1 [==============================] - 0s 3ms/step - loss: 19059.0879
Epoch 299/5000
1/1 [==============================] - 0s 4ms/step - loss: 19047.1289
Epoch 300/5000
1/1 [==============================] - 0s 3ms/step - loss: 19035.2129
Epoch 301/5000
1/1 [==============================] - 0s 3ms/step - loss: 19023.

1/1 [==============================] - 0s 3ms/step - loss: 18130.1523
Epoch 387/5000
1/1 [==============================] - 0s 3ms/step - loss: 18120.7109
Epoch 388/5000
1/1 [==============================] - 0s 4ms/step - loss: 18111.2949
Epoch 389/5000
1/1 [==============================] - 0s 3ms/step - loss: 18101.8906
Epoch 390/5000
1/1 [==============================] - 0s 3ms/step - loss: 18092.5117
Epoch 391/5000
1/1 [==============================] - 0s 3ms/step - loss: 18083.1484
Epoch 392/5000
1/1 [==============================] - 0s 3ms/step - loss: 18073.8105
Epoch 393/5000
1/1 [==============================] - 0s 4ms/step - loss: 18064.4844
Epoch 394/5000
1/1 [==============================] - 0s 3ms/step - loss: 18055.1836
Epoch 395/5000
1/1 [==============================] - 0s 3ms/step - loss: 18045.8945
Epoch 396/5000
1/1 [==============================] - 0s 3ms/step - loss: 18036.6289
Epoch 397/5000
1/1 [==============================] - 0s 4ms/step - loss: 18027.

...

1/1 [==============================] - 0s 2ms/step - loss: 9187.6270
Epoch 4497/5000
1/1 [==============================] - 0s 2ms/step - loss: 9184.5938
Epoch 4498/5000
1/1 [==============================] - 0s 2ms/step - loss: 9187.3838
Epoch 4499/5000
1/1 [==============================] - 0s 2ms/step - loss: 9189.1992
Epoch 4500/5000
1/1 [==============================] - 0s 2ms/step - loss: 9186.2588
Epoch 4501/5000
1/1 [==============================] - 0s 2ms/step - loss: 9190.0586
Epoch 4502/5000
1/1 [==============================] - 0s 3ms/step - loss: 9191.4277
Epoch 4503/5000
1/1 [==============================] - 0s 2ms/step - loss: 9236.2080
Epoch 4504/5000
1/1 [==============================] - 0s 3ms/step - loss: 9183.7510
Epoch 4505/5000
1/1 [==============================] - 0s 2ms/step - loss: 9189.0684
Epoch 4506/5000
1/1 [==============================] - 0s 2ms/step - loss: 9187.7969
Epoch 4507/5000
1/1 [==============================] - 0s 2ms/step - loss: 9188.0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 2ms/step - loss: 9171.7412
Epoch 4764/5000
1/1 [==============================] - 0s 2ms/step - loss: 9168.0830
Epoch 4765/5000
1/1 [==============================] - 0s 2ms/step - loss: 9217.9785
Epoch 4766/5000
1/1 [==============================] - 0s 2ms/step - loss: 9170.1875
Epoch 4767/5000
1/1 [==============================] - 0s 2ms/step - loss: 9225.0234
Epoch 4768/5000
1/1 [==============================] - 0s 2ms/step - loss: 9167.8809
Epoch 4769/5000
1/1 [==============================] - 0s 2ms/step - loss: 9169.2520
Epoch 4770/5000
1/1 [==============================] - 0s 2ms/step - loss: 9172.8496
Epoch 4771/5000
1/1 [==============================] - 0s 2ms/step - loss: 9233.9941
Epoch 4772/5000
1/1 [==============================] - 0s 2ms/step - loss: 9183.3262
Epoch 4773/5000
1/1 [==============================] - 0s 2ms/step - loss: 9172.0781
Epoch 4774/5000
1/1 [==============================] - 0s 2ms/step - loss: 9173.6

1/1 [==============================] - 0s 2ms/step - loss: 9170.5273
Epoch 4860/5000
1/1 [==============================] - 0s 2ms/step - loss: 9187.7773
Epoch 4861/5000
1/1 [==============================] - 0s 2ms/step - loss: 9165.2783
Epoch 4862/5000
1/1 [==============================] - 0s 2ms/step - loss: 9165.8057
Epoch 4863/5000
1/1 [==============================] - 0s 2ms/step - loss: 9164.0879
Epoch 4864/5000
1/1 [==============================] - 0s 2ms/step - loss: 9162.3896
Epoch 4865/5000
1/1 [==============================] - 0s 2ms/step - loss: 9164.4277
Epoch 4866/5000
1/1 [==============================] - 0s 2ms/step - loss: 9166.3545
Epoch 4867/5000
1/1 [==============================] - 0s 2ms/step - loss: 9166.9248
Epoch 4868/5000
1/1 [==============================] - 0s 2ms/step - loss: 9162.0928
Epoch 4869/5000
1/1 [==============================] - 0s 2ms/step - loss: 9163.3740
Epoch 4870/5000
1/1 [==============================] - 0s 2ms/step - loss: 9161.4

1/1 [==============================] - 0s 2ms/step - loss: 9162.2207
Epoch 4956/5000
1/1 [==============================] - 0s 2ms/step - loss: 9158.8418
Epoch 4957/5000
1/1 [==============================] - 0s 2ms/step - loss: 9157.0361
Epoch 4958/5000
1/1 [==============================] - 0s 2ms/step - loss: 9156.0156
Epoch 4959/5000
1/1 [==============================] - 0s 3ms/step - loss: 9208.4082
Epoch 4960/5000
1/1 [==============================] - 0s 2ms/step - loss: 9160.3203
Epoch 4961/5000
1/1 [==============================] - 0s 2ms/step - loss: 9167.9824
Epoch 4962/5000
1/1 [==============================] - 0s 2ms/step - loss: 9165.4580
Epoch 4963/5000
1/1 [==============================] - 0s 2ms/step - loss: 9161.2451
Epoch 4964/5000
1/1 [==============================] - 0s 2ms/step - loss: 9156.6484
Epoch 4965/5000
1/1 [==============================] - 0s 2ms/step - loss: 9155.3301
Epoch 4966/5000
1/1 [==============================] - 0s 3ms/step - loss: 9158.1

In [10]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input.npy', X_test)
np.save('output.npy', pred_v)

92/92 [==============================] - 0s 364us/step


In [11]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df['Right Value - Mean Other Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - O)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)

pred_bhv['right'] = pd.get_dummies(pred_bhv['ch'])[2]
pred_bhv.to_csv('Kr2011_complete.csv')